## Skinning

This notebook will allow you to learn about what computer animators use behind the scenes to create animations for movies and video games all the time, _linear blend skinning_.


## 1. Basics
Let's start by importing all the python packages we will be using throughout this project. This is very simply done by running the below code block.

In [3]:
import igl
import polyscope as ps
import numpy as np
from keyframe_animate import keyframe_animate

### comment out below as they have solutions
from scale import scale
from scale import scale
from translate import translate
from rotate import rotate
from lerp import lerp

Next we will try to read in a triangle mesh. A triangle mesh is the basic tool we use to describe a shape to a computer. It can be described with two very simple lists (or as we say in computer science, _arrays_).

`X` is a list of vertex positions. These are where each of the triangle meshes points are located.

`F` is a list of triangle indices. Each triangle simply contains three numbers that say which of the vertices in `X` are used to create this triangle.

We can do this first on a simple triangle, below we have a simple triangle represented with a triangle mesh. `X` contains the positions of the three points, and `F` here contains which points are used in `X` to compose each triangle! Because there's only one triangle, `F` is very simple and has only one row. 

In [4]:
X = [[0, 0, 0],
    [1, 0, 0], 
    [1, 1, 0.]]
F = [[0, 1, 2]]

ps.init()
ps.register_surface_mesh("mesh", np.array(X), np.array(F), edge_width=1)
ps.register_point_cloud("vertices", np.array(X), radius=0.05, color=[0, 0, 0])
ps.reset_camera_to_home_view()
ps.show()


For now, that annoying `ps` code is just the code we use to tell the computer to display the triangle mesh. To do this, we've made us of our installed polyscope library!

While the above is super simple, we can easily extend this to extremely complicated `X` and `F` to describe complex 3D shapes. The below shows a triangle mesh of a snail, that we are reading in from the `data` directory I have provided.

In [5]:
[X, _, _, F, _, _]  = igl.read_obj("./data/snail.obj")
ps.init()
ps.register_surface_mesh("mesh", X, F, edge_width=1)
ps.register_point_cloud("vertices", X, radius=0.001, color=[0, 0, 0])
ps.reset_camera_to_home_view()
ps.show()

Let's work with something less complicated for now, a 2D meshes

In [25]:
[X, _, _, F, _, _]  = igl.read_obj("./data/cthullu.obj")
ps.init()
ps.register_surface_mesh("mesh", X, F, edge_width=1)
ps.register_point_cloud("vertices", X, radius=0.001, color=[0, 0, 0])
ps.reset_camera_to_home_view()
ps.show()

## 2. Mixing Quantities with "Linear Interpolation"

Imagine I have two quantities, `q1`, which represents the amount of milk in my white chocolate (let's say `100` mL), and `q2`, which represents the amount of milk in my dark chocolate (let's say `40mL`). 


But to make my top secret chocolate recipe, I don't use straight up white chocolate or straight up dark chocolate, I use my own custom amount of milk that I'm not telling anyone about!

Unfortunately, at the chocolatier conference I make the mistake of telling a Hershey's representative that my chocolate is exactly a third (`s=1/3`)  of the way between white chocolate and dark chocolate.

The next day the top news on the press is : "Otman's chocolate revealed to contain `80mL` of of milk! Everyone can make it at home now!". My business is ruined.

How did Hershey's figure out my top secret amount of milk? They used *linear interpolation* of course!

Linear interpolation takes two quantities $q_1$ $q_2$, as well as a fraction $s$  I use to determine how much of each I want, and produces a mixture of both quantities!

$$
q_{mix} = q_1 s + q_2 (1 - s)
$$

Your first task is to edit the function below to do linear interpolation... we're going to be interpolating our own quantities soon enough to make animations! Make sure the predicted amount of milk in Otman's chocolate matches what Hershey's predicted!



In [28]:
def lerp(q1, q2, s):
    # Replace the code below so that we can linearly interpolate matrix A1 and A2, at timestep s, with a maximum timestep of t
    q = q1

    q = q1 * (1-s) + q2 * ( s)
    return q

q1 = 100
q2 = 40

q = lerp(q1, q2, 1/3)
print("The amount of milk in Otman's chocolate, according to Hershey's, is", q, "mL")


The amount of milk in Otman's chocolate, according to Hershey's, is 80.0 mL


In graphics and animation _interpolation_ is in particular important for _keyframe animation_.  This is an animation method that lets a user specify two different poses, and the keyframe animation algorithm smoothly transitions between the two poses. This is done by interpolating the poses at each time step, and then displaying the result.

##  Controlling Virtual Characters

Editing 3D scenes is notoriously difficult. Picture all the virtual characters in a normal movie or video game. For example, look at this nice picture of Woody!

<img src="./images/woody.jpg" alt="drawing" width="400"/>
<br>

Looks so nice and polished! Well imagine being the animator that created this picture! They had to make sure the hat is oriented on his head the way we expect, that his smile looks natural and not weird, that his arms are crossed over in a way that feels intuitive... and they have to do all this virtually on their computer!
They have to open their 3D modelling software and look at the mesh of woody

<img src="./images/Woody-Mesh-300x259.jpg" alt="drawing" width="400"/>
<br>

And then they somehow have to edit and pose the positions of each and every one of these potentially thousands of vertices!!!
Sounds hard, right? Well, it is! But luckily we have tools like _linear_blend_skinning_ to help us out!

This first important question we have to ask ourselves though, is what does it mean to ask our computer to "move" or "pose" an object?



## Affine Matrices

We're going to see that lots of the most important kinds of motions an animator might want, can be represented by a so-called _affine matrix_. You're going to learn a lot about these if you take any STEM degree in university, but we're going to tell you the basics here.

A matrix is just a group of numbers organized into a grid! For example, the matrix below is a 2x3 matrix, because it has 2 rows and 3 columns.

$$
\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6
\end{bmatrix}
$$
Organizing these numbers into matrix is _super_ important in math and literally _any_ science, because it allows you to represent a _lot_ of information in a _very_ compact way. For example, the matrix above could represent the system of these equations:

$$
\begin{align*}
x + 2y = 3 \\
4x + 5y = 6
\end{align*}
$$
You might be used to have your teachers asking you to figure out x and y, given the above two equations. You probably know how annoying it is to be asked that.
Luckily, if you convert it to a matrix form like the above, you can just use a computer to solve it for you :D
... but why the heck am I telling you this? I was just telling you about how we move objects in space, and now I'm giving you nightmare flashbacks from your past or future math classes!

Well, it turns out that we can write many geometric motions as a matrix as well!!! Some specific matrices have a very intuitive understanding. The matrices we're going to be dealing with in this project will always be 3x4, and will look like this:

$$
A = \begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14} \\
a_{21} & a_{22} & a_{23} & a_{24} \\
a_{31} & a_{32} & a_{33} & a_{34}
\end{bmatrix}
$$

By changing each of the $a$ values in the matrix above, we can achieve different motions!


## 3. Translation Matrix
Maybe the simplest geometric motion we can think about is a "translation". A translation, in math, means just sliding and object.
So when I say, translate this object to the left by 2 meters, all I'm saying is slide it to the left two meters. You're not allowed to rotate it, or expand it or anything else.

It turns out translations have this very simple matrix representation:
$$
A = \begin{bmatrix}
1 & 0 & 0 & t_x \\
0 & 1 & 0 & t_y \\
0 & 0 & 1 & t_z
\end{bmatrix}
$$
Where $t_x$, $t_y$, and $t_z$ are the amount we want to slide the object in the x, y, and z directions respectively. Note that the diagonal entries of ones are super important!

Change the function below to return the translation matrix for a given translation vector `t` with three entries `t[0] = t_x`, `t[1] = t_y` and `t[2]=t_z`. Once you're happy with your function, feel free to change the translation vector `t1` and `t2` to change the kind of motions your mesh can take.

The function `keyframe_animate`, takes as input our mesh,  and two 3x4 affine matrices, and performs the linear interpolation function we defined above on them, so that we can smoothly transition from one to the next!

In [27]:
def translate(t):
    # Create an affine matrix A 3x4 affine matrix that represents a translation operation
    # Given a list of 3 entries t1
    A = [[1, 0, 0, t[0]],
         [0, 1, 0, t[1]],
         [0, 0, 1, t[2]]]
    return np.array(A)

t1 = [0, 0, 0]
t2 = [0, 1, 0]
A3 = translate(t1)
A4 = translate(t2)

keyframe_animate(X, F, A3, A4, lerp, 50)
keyframe_animate(X, F, A4, A3, lerp, 50)

## 4. Scaling
The next geometric motion we care about is "scaling"! Scaling is just the fancy name we've given to the concept of making things bigger/smaller.

The affine matrix for scaling is super simple as well! It looks like this:
$$
A = \begin{bmatrix}
s_x & 0 & 0 & 0 \\
0 & s_y & 0 & 0 \\
0 & 0 & s_z & 0
\end{bmatrix}
$$

Where $s_x$, $s_y$, and $s_z$ are the amount we want to make the object bigger or smaller in the x, y, and z directions respectively.

Note that you can make something bigger by using a number for $s_x$, $s_y$ and $s_z$ bigger than 1, and smaller by using a number smaller than 1.

Note that we can assign different scaling in either direction, $s_x$ can be different from $s_y$ and $s_z$.
When $s_x = s_y = s_z$, we call this a _uniform_ scaling. When they are different, we call this a _non-uniform_ scaling.

Change the function below to return the scaling matrix for a given scaling vector `s` with three entries `s[0] = s_x`, `s[1] = s_y` and `s[2]=s_z`. Once you're happy with your function, feel free to change the scaling vector `s1` and `s2` to change the kind of motions your mesh can take. Be sure to investigate what non-uniform scaling is.


In [30]:
def scale(s):
    # Create an affine matrix A 3x4 affine matrix that represents a scaling operation
    # Given a list of 3 entries s
    A = [[s[0], 0, 0, 0],
         [0, s[1], 0, 0],
         [0, 0, s[2], 0]]
    return np.array(A)
    
s1 = [1, 1, 1]
s2 = [1, 3, 1]
A1 = scale(s1)
A2 = scale(s2)

keyframe_animate(X, F, A1, A2, lerp, 50)
keyframe_animate(X, F, A2, A1, lerp, 50)

## 5. Rotations
The next geometric motion we care about is "rotation"! A rotation lets you turn your object to face a different direction.
You've probably heard of a rotation before, and now we can tell you how to represent this as an affine matrix as well!

The affine matrix for 2D rotation is a bit more complicated than the previous ones. But still not too bad! It looks like this:

$$
A = \begin{bmatrix}
cos(\theta)  & - sin(\theta) & 0 & 0 \\
sin(\theta) & cos(\theta)  & 0 & 0 \\
0 & 0 & 1 & 0
\end{bmatrix}
$$

Where $\theta$ is the amount we want to rotate the object. Note that the above formula is only for a 2D rotation, where you're rotating the shape `about` the z-axis. More complicated rotations are possible in 3D, about arbitrary axes, but we won't be dealing with those in this project for now.

In [34]:
def rotate(theta):
    # Create an affine matrix A 3x4 affine matrix that represents a rotation operation
    # Given a list of 3 entries r
    A = [[np.cos(theta), -np.sin(theta), 0, 0],
         [np.sin(theta), np.cos(theta), 0, 0],
         [0, 0, 1, 0]]
    return np.array(A)

# r1 = [0, 0, 0]
# r2 = [0, 0, np.radians(90)]
r1 = 0
r2 = np.radians(90)
A5 = rotate(r1)
A6 = rotate(r2)
keyframe_animate(X, F, A5, A6, lerp,  50)
keyframe_animate(X, F, A6, A5, lerp,  50)

Hold on just one second... did you notice that?? If possible, bring the code on the right side of the screen, and polyscope to the left, and rerun the code above so you can get a closer look...

Doesn't it look like the mesh is _shrinking_ and _scaling_ as it's rotating?? It's just for a second... maybe I'm crazy. Let's try something a little different. Let's try to make it rotate from 0-180 degrees in one go!

In [43]:
r3 =  -np.radians(180)
A7 = rotate(r3)
keyframe_animate(X, F, A5, A7, lerp,  50)
keyframe_animate(X, F, A7, A5, lerp,  50)

No, something for sure is _definitely_ going on. It looks like the mesh shrinks and then disappears and then flips on the other side!!! Why doesn't it just rotate like we expect it to?

Why does this happen??? The reason is that our _linear interpolation_ does not properly interpolate rotations the way we expect. As a bonus, try to show when interpolating between a 0-rotation and a 180-rotation will give you something bad in the middle.


This is a well known problem in graphics and the effect you see is given the name "the Candy Wrapper Effect", and can lead to disastrous results when artists try to rotate parts of their shape!

<img src="./images/candy-wrapper.png" alt="drawing" width="400"/>
<br>

### 6. Spherical Linear Interpolation

The solution to this problem is to use a different kind of interpolation, called _spherical linear interpolation_ or _slerp_ for short. This is a more complicated kind of interpolation that is specifically designed to interpolate rotations in a way that doesn't give you the candy wrapper effect.

The idea here is instead of interpolating the rotation matrices directly, like we were doing in lerp, we're going to be interpolating the angles that define the rotation matrices, and then converting these angles back into rotation matrices.

In [46]:
def slerp(r1, r2, s):
     # Interpolate the angles r1 and r2 with fraction s , and then build a rotation matrix off that interpolated rotation.
    r1 = np.array(r1)
    r2 = np.array(r2)
    r = lerp(r1, r2, s)
    R = rotate(r)
    return R

r1 = 0
r2 = np.radians(-180)
keyframe_animate(X, F, r1, r2, slerp,  50)
keyframe_animate(X, F, r2, r1, slerp,  50)

Much better, right? Our shape correctly rotates to the 180 mark finally :D.

### 7. Bonus : Linear Blend Skinning
The last thing we're going to talk about is _linear blend skinning_ or _LBS_ for short. This is a technique that is used _all the time_ in computer animation to help artists design movement.

Instead of having an artist specify motion vertex by vertex, we're going to group vertices of the character together, and only specify motion for each of these groups.

This is the formula for linear blend skinning, for the 3D position of a single vertex $x$:

$$
x =  \sum_{j=0}^{b} w_{j} T_j {x_0}
$$

Don't worry, this looks complicated, but it's not so bad. Here's what it means:

- $x$ is the position of the vertex after it's been moved by the skinning algorithm. This is the desired output.
- $x_0$ is the position of the vertex in the rest pose, the one we've been working with so far.
- $T_j$ is the j'th affine matrix that specifies some motion.
- $w_j$ is the skinning weight , aka, the amount of influence the j-th affine matrix has on the vertex.

If you've never seen it before, the $\sum$ symbol just means "add up all the terms from $j=0$ to $b$". So the formula is just saying "add up all the terms from $j=0$ to $b$".

The idea is that we're going to have a bunch of different affine matrices $T_j$, and a bunch of different skinning weights $w_j$, and we're going to add up all the different motions that each of these matrices specifies, weighted by the skinning weights.

In [2]:
def lbs(X0, W, T):
    # Perform linear blend skinning on the mesh X0, with a list of b skinning weights W, and a list of b affine matrices T
    b = W.shape[1]
    X = np.zeros((X0.shape[0], 3))

    ### sum over all affine matrices b, their contributions
    for j in range(b):
        xd =  W[:, j] * (T[j] @ X0.T)
        X += xd.T
    return X

In [4]:
import polyscope as ps
import numpy as np
from load_rig_anim import load_rig_anim


ps.remove_all_groups()
ps.remove_all_structures()


rig_file = "./data/michelle/rig.json"
anim_file = "./data/michelle/anim.json"


[X, F, W, Prel] = load_rig_anim(rig_file, anim_file)

X01 = np.hstack((X, np.ones((X.shape[0], 1))))
for t in range(Prel.shape[0]):
    p = Prel[t, :, :, :]

    X = lbs(X01, W, p)

    if t == 0:
        ps.init()
        mesh = ps.register_surface_mesh("mesh", X, F, edge_width=1)
        ps.reset_camera_to_home_view()
    else:
        mesh.update_vertex_positions(X)
        ps.frame_tick()
    mesh.update_vertex_positions(X)
    ps.frame_tick()
